In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import itertools

from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
df = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [5]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}
base_skin_dir = '../input/skin-cancer-mnist-ham10000'

In [6]:
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

In [7]:
df['path'] = df['image_id'].map(imageid_path_dict.get)
df['cell_type'] = df['dx'].map(lesion_type_dict.get)
df['cell_type_idx'] = pd.Categorical(df['cell_type']).codes

df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,../input/skin-cancer-mnist-ham10000/ham10000_i...,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,../input/skin-cancer-mnist-ham10000/ham10000_i...,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,../input/skin-cancer-mnist-ham10000/ham10000_i...,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,../input/skin-cancer-mnist-ham10000/ham10000_i...,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,../input/skin-cancer-mnist-ham10000/ham10000_i...,Benign keratosis-like lesions,2


In [8]:
df.isna().sum()

lesion_id         0
image_id          0
dx                0
dx_type           0
age              57
sex               0
localization      0
path              0
cell_type         0
cell_type_idx     0
dtype: int64

In [9]:
df['image'] = df['path'].map(lambda x: np.asarray(Image.open(x).resize((100,75))))

In [10]:
df['age'].fillna((df['age'].mean()), inplace=True)

<ipython-input-10-6c9f6c8fc82c>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna((df['age'].mean()), inplace=True)


In [11]:
features=df.drop(columns=['cell_type_idx'],axis=1)
target=df['cell_type_idx']

In [12]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.20,random_state=1234)

In [13]:
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [14]:
print("Original labels:", y_train_o)
print("Shape of y_train:", y_train_o.shape)

Original labels: 8442    4
2908    1
8909    4
58      2
2960    1
       ..
664     2
7540    4
7221    4
1318    5
8915    4
Name: cell_type_idx, Length: 8012, dtype: int8
Shape of y_train: (8012,)


In [15]:
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)
print("Shape of y_train:", y_train.shape)

Shape of y_train: (8012, 7)


In [16]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2)

print("Shape of y_train:", y_train.shape)

Shape of y_train: (7210, 7)


In [17]:
# Reshape image in 3 dimensions (height = 75px, width = 100px , canal = 3)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
#x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

In [18]:
print(f"x_train shape before resampling: {x_train.shape}")
print("Shape of y_train:", y_train.shape)

x_train shape before resampling: (7210, 75, 100, 3)
Shape of y_train: (7210, 7)


In [19]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
#x_validate = x_validate.reshape(x_validate.shape[0], -1)
print(f"Flattened X_class shape: {x_train.shape}")
print("Shape of y_train:", y_train.shape)

Flattened X_class shape: (7210, 22500)
Shape of y_train: (7210, 7)


In [20]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

class COSHySMOTE:
    def __init__(self, target_distribution=None, cluster_sizes=None, random_state=None):
        """
        Initialize COSHySMOTE.

        Parameters:
        - target_distribution: dict, desired class distribution after resampling. Example: {0: 10, 1: 20}.
        - cluster_sizes: dict, number of clusters to use per class. Example: {0: 5, 1: 3}.
        - random_state: int, random state for reproducibility.
        """
        self.target_distribution = target_distribution
        self.cluster_sizes = cluster_sizes
        self.random_state = random_state

    def fit_resample(self, X, y):
        """
        Fit and resample the dataset.

        Parameters:
        - X: numpy array, feature matrix.
        - y: numpy array, labels.

        Returns:
        - X_resampled: numpy array, resampled feature matrix.
        - y_resampled: numpy array, resampled labels.
        """
        unique_classes, class_counts = np.unique(y, return_counts=True)
        original_distribution = dict(zip(unique_classes, class_counts))

        print(f"Original class distribution: {original_distribution}")

        if self.target_distribution is None:
            raise ValueError("Target distribution must be specified.")

        X_resampled, y_resampled = [], []
        
        for cls in unique_classes:
            class_indices = np.where(y == cls)[0]
            X_class = X[class_indices]
            n_clusters = self.cluster_sizes.get(cls, min(len(X_class), 5))  # Default to min(len, 5)

            # Clustering
            print(f"Clustering {len(X_class)} samples into {n_clusters} clusters for class {cls}.")
            kmeans = KMeans(n_clusters=n_clusters, random_state=self.random_state)
            cluster_labels = kmeans.fit_predict(X_class)
            
            # Retain one sample per cluster
            retained_samples = self._retain_samples(X_class, cluster_labels, kmeans)
            X_resampled.extend(retained_samples)
            y_resampled.extend([cls] * len(retained_samples))
            print(f"Clusters: {n_clusters}, Retained Samples for class {cls}: {len(retained_samples)}")

            # Generate synthetic samples if needed
            target_count = self.target_distribution.get(cls, len(retained_samples))
            if target_count > len(retained_samples):
                synthetic_count = target_count - len(retained_samples)
                synthetic_samples = self._generate_synthetic_samples(
                    X_class, retained_samples, cluster_labels, kmeans, synthetic_count
                )
                X_resampled.extend(synthetic_samples)
                y_resampled.extend([cls] * len(synthetic_samples))
                print(f"Generated {len(synthetic_samples)} synthetic samples for class {cls}.")

        # Verify final distribution
        resampled_distribution = dict(zip(*np.unique(y_resampled, return_counts=True)))
        print(f"Resampled class distribution: {resampled_distribution}")

        return np.array(X_resampled), np.array(y_resampled)

    def _retain_samples(self, X_class, cluster_labels, kmeans):
        """
        Retain one sample per cluster.

        Parameters:
        - X_class: numpy array, samples of the class.
        - cluster_labels: numpy array, cluster assignments.
        - kmeans: fitted KMeans object.

        Returns:
        - List of retained samples.
        """
        retained_samples = []
        for cluster_id in np.unique(cluster_labels):
            cluster_indices = np.where(cluster_labels == cluster_id)[0]
            cluster_center = kmeans.cluster_centers_[cluster_id]
            closest_idx = cluster_indices[
                np.argmin(np.linalg.norm(X_class[cluster_indices] - cluster_center, axis=1))
            ]
            retained_samples.append(X_class[closest_idx])
        return retained_samples

    def _generate_synthetic_samples(self, X_class, retained_samples, cluster_labels, kmeans, synthetic_count):
        """
        Generate synthetic samples.

        Parameters:
        - X_class: numpy array, samples of the class.
        - retained_samples: list, retained samples.
        - cluster_labels: numpy array, cluster assignments.
        - kmeans: fitted KMeans object.
        - synthetic_count: int, number of synthetic samples to generate.

        Returns:
        - List of synthetic samples.
        """
        synthetic_samples = []
        nearest_neighbors = NearestNeighbors(n_neighbors=5).fit(X_class)

        for _ in range(synthetic_count):
            cluster_id = np.random.choice(np.unique(cluster_labels))
            cluster_indices = np.where(cluster_labels == cluster_id)[0]
            random_idx = np.random.choice(cluster_indices)
            base_sample = X_class[random_idx]

            # Find neighbors within the cluster
            neighbors = nearest_neighbors.kneighbors(base_sample.reshape(1, -1), return_distance=False)[0]
            neighbor_idx = np.random.choice(neighbors)
            neighbor_sample = X_class[neighbor_idx]

            # Interpolate to create a synthetic sample
            alpha = np.random.uniform(0, 1)
            synthetic_sample = base_sample + alpha * (neighbor_sample - base_sample)
            synthetic_samples.append(synthetic_sample)

        return synthetic_samples


In [21]:
#from sklearn.decomposition import PCA

#pca = PCA(n_components=500)  # Retain 500 principal components
#x_train = pca.fit_transform(x_train)

In [ ]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate class counts
class_counts = Counter(np.argmax(y_train, axis=1))  # Use np.argmax if y_train is one-hot encoded
classes = list(class_counts.keys())  # Get all unique classes

# Determine total samples for balancing
total_samples = max(class_counts.values())  # Use the maximum count as the target for oversampling

# Desired samples for each class (make all classes proportional)
target_distribution = {cls: total_samples for cls in classes}

# Adjust retained samples and cluster sizes for each class
retained_samples = {cls: max(1, min(100, class_counts[cls] // 1000)) for cls in classes}
cluster_sizes = {cls: retained_samples[cls] for cls in classes}

# COSHySMOTE resampling
coshysmote = COSHySMOTE(
    target_distribution=target_distribution,
    cluster_sizes=cluster_sizes,
    random_state=42
)
X_resampled, y_resampled = coshysmote.fit_resample(x_train, np.argmax(y_train, axis=1))  # Convert one-hot to integer labels

# Check new resampled distribution
resampled_class_counts = Counter(y_resampled)
print("Resampled class distribution:", dict(resampled_class_counts))

# Visualize the resampled class distribution
plt.figure(figsize=(10, 5))
sns.countplot(x=y_resampled, palette="tab10")  # Use tab10 for more class colors
plt.title("Resampled Class Distribution After COSHySMOTE")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()


In [ ]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate class counts
class_counts = Counter(np.argmax(y_train, axis=1))  # Use np.argmax if y_train is one-hot encoded
classes = list(class_counts.keys())  # Get all unique classes

# Determine total samples for balancing
total_samples = max(class_counts.values())  # Use the maximum count as the target for oversampling

# Desired samples for each class (make all classes proportional)
target_distribution = {cls: total_samples for cls in classes}

# Adjust retained samples and cluster sizes for each class
retained_samples = {cls: max(1, min(100, class_counts[cls] // 1000)) for cls in classes}
cluster_sizes = {cls: retained_samples[cls] for cls in classes}

# COSHySMOTE resampling
coshysmote = COSHySMOTE(
    target_distribution=target_distribution,
    cluster_sizes=cluster_sizes,
    random_state=42
)
X_resampled, y_resampled = coshysmote.fit_resample(x_train, np.argmax(y_train, axis=1))  # Convert one-hot to integer labels

# Check new resampled distribution
resampled_class_counts = Counter(y_resampled)
print("Resampled class distribution:", dict(resampled_class_counts))

# Visualize the resampled class distribution
plt.figure(figsize=(10, 5))
sns.countplot(x=y_resampled, palette="tab10")  # Use tab10 for more class colors
plt.title("Resampled Class Distribution After COSHySMOTE")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()


In [ ]:
wait


In [ ]:
from collections import Counter
# Adjust target samples for balanced distribution
total_majority = len(y_train[y_train == 0])
total_minority = len(y_train[y_train == 1])

# Desired total samples for each class
desired_minority_samples = total_majority // 2  # Keep the minority class size proportional to the majority
desired_majority_samples = desired_minority_samples

# Adjust retained samples and target synthetic samples
retained_samples = {
    0: min(100, total_majority // 1000),
    1: min(50, total_minority // 10)
}

target_samples = {
    0: desired_majority_samples,
    1: desired_minority_samples
}

# Apply COSHySMOTE
coshysmote = COSHySMOTE(
    target_samples,
    retained_samples,
    random_state=42
)
X_resampled, y_resampled = coshysmote.fit_resample(x_train, y_train)

# Check new resampled distribution
print("Resampled class distribution:", dict(Counter(y_resampled)))

# Visualize the resampled class distribution
plt.figure(figsize=(10, 5))
sns.countplot(x=y_resampled, palette=["#0101DF", "#DF0101"])
plt.title("Resampled Class Distribution After COSHySMOTE")
plt.show()



Original class distribution: {0.0: 43260, 1.0: 7210}
Clustering 43260 samples into 43 clusters for class 0.0.


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/numpy/core/multiarray.py", line 346, in where
    @array_function_from_c_func_and_dispatcher(_multiarray_umath.where)
KeyboardInterrupt: 


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/numpy/core/multiarray.py", line 346, in where
    @array_function_from_c_func_and_dispatcher(_multiarray_umath.where)
KeyboardInterrupt: 


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Assuming `y_resampled` contains the labels after balancing
# Relabel class names if needed (for clarity)
class_names = {
    0: 'nv',
    1: 'mel',
    2: 'bkl',
    3: 'vasc',
    4: 'df',
    5: 'akiec',
    6: 'bcc'
}

# Map numeric labels to class names
y_resampled_named = [class_names[label] for label in y_resampled]

# Create a pandas DataFrame for visualization
df = pd.DataFrame({'Class': y_resampled_named})
class_counts = df['Class'].value_counts()

# Plot class distribution
plt.figure(figsize=(10, 6))
sns.barplot(x=class_counts.index, y=class_counts.values, palette="viridis")
plt.title('Class Distribution After Balancing (HAM10000)', fontsize=16)
plt.xlabel('Class', fontsize=14)
plt.ylabel('Number of Samples', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.show()
